### mod_sql모듈
1. mod_sql오듈 로드
2. class를 생성
    - 대면 : host(172.16.12.149) ubion password (1234)
    - 비대면 : host(darkpreist.iptime.org) user(ubion) passresd
3. select문을 만들어서 excutedAll 함수를 이용해서 결과값 리턴
    - corona데이터를 select
    - 데이터의 개수는 10개
4. inser문을 이용하여 데이터삽입
    - user_list테이블에 id와 password를입력
    - id는 자기 자신의 이름, password 랜덤하게 입력
5. commit(), close() 해서 데이터베이스와의 접속 종료

In [1]:
import mod_sql  as ms
import imp
imp.reload(ms)

C:\Users\msi\AppData\Local\Temp\ipykernel_21072\1045150083.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


<module 'mod_sql' from 'c:\\ubion\\221227\\mod_sql\\__init__.py'>

In [34]:
## 클래스 생성
sql_class = ms.Database("172.16.12.149",'ubion','1234')

In [4]:
sql = """
        select  
        *
        from 
        corona
      """
sql_class.excuteAll(sql,_limit=10)

,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,2022-06-08 09:09:05.982,24305,18188200,904,20220608,00:00,None,,
1,2022-06-07 09:09:00.897,24299,18174842,903,20220607,00:00,2022-06-08 09:10:36.846,,
2,2022-06-06 09:00:06.734,24279,18168670,902,20220606,00:00,2022-06-08 09:10:50.441,,
3,2022-06-05 08:53:19.426,24258,18163648,901,20220605,00:00,2022-06-08 09:11:04.758,,
4,2022-06-04 08:56:49.219,24238,18153814,900,20220604,00:00,2022-06-08 09:11:26.303,,
5,2022-06-03 09:08:18.729,24229,18141775,899,20220603,00:00,2022-06-08 09:11:38.938,,
6,2022-06-02 08:58:19.746,24212,18129236,898,20220602,00:00,2022-06-08 09:11:51.985,,
7,2022-06-01 09:07:30.461,24197,18119345,897,20220601,00:00,2022-06-08 09:12:05.316,,
8,2022-05-31 08:55:39.977,24176,18103577,896,20220531,00:00,2022-06-04 08:59:23.637,,
9,2022-05-30 09:01:30.847,24167,18086392,895,20220530,00:00,2022-06-04 08:59:37.203,,


In [5]:
insert_sql = """
        insert into
        user_list(id, password)
        values
        (%s,%s)
    """    
sql_class.execute(insert_sql,_values=['기지훈','1234'])


()

In [6]:
sql_class.commit()

' commit complete'

In [12]:
delete_sql  = """
                delete
                from user_list
                where
                `id` = %s
    """
sql_class.execute(delete_sql,_values=['test'])
sql_class.commit()

SyntaxError: invalid syntax (623260662.py, line 1)

In [20]:
## user_list 테이블에서 password가 1234인 데이터를 출력 
## values를 사용하여
sql = """ 
        select *
        from user_list
        where `password` = %s 
        """
values = ['1234']
sql_class.excuteAll(sql,values)

,id,password
0,hong,1234
1,기지훈,1234
2,김정우,1234
3,서선우,1234
4,전성표,1234
5,홍성일,1234


In [3]:
sql_class.close()

'Database close'

### data의 확인
1. id, password의 값을 받아오는 작업(input)
2. user_list 테이블에서 받아온 id, password의 값이 존재하는지 체크
3. 존재한다면 '성공',존재하지않으면'실패' 메세지를 리턴

In [4]:
sql_class = ms.Database("172.16.12.149",'ubion','1234')

In [42]:
_id = input("id :")
_pass = input("password :")
sql = """
        select *
        from user_list
        where `id` = %s  
        and 
        `password` = %s
    """
result = sql_class.execute(sql,[_id,_pass])
## result 값을 예상
## 성공 - > id와 password의 값이 데이터 안에 존재
## -> result 안에 데이터의 형태는 [ {'id' : '기지훈'},{'password' : '1234'}]
## -> 성공시 리스트 안에  항목은 1 개 이유는? -> id가 primary key이기 때뮨에
##  유니크 속성과 not null의 속성을 가진다.
## id로 검색하는 경우에는 1개의 데이터 이상을 가질 수 없다.
## 실패 -> id 와 password의 값이 존재하지 않는 경우 id와 password중 하나만 틀려도 실패
## -> result 안에 데이터의 형태는 () 빈 튜플이 나옴

## 데이터의 존재 유무
# case1
if result == ():
    print("실패")
else :
    print("성공")

#case2
if _id == result[0]['id'] and _pass == result[0]['password']:
    print("성공")
else:
    print("실패")

# case3
if result:
    print('성공')
else :
    print('실패')




성공
성공
성공


### api형식의 데이터를 보내기
1. id, password의 데이터가 존재하면(로그인이 성공하면)
2. 성공시에는 corona 데이터를 데이터프레임으로 출력
3. 실패시에는 '로그인 실패'

In [58]:
_id = input("id :")
_pass = input("password :")
sql = """
        select *
        from user_list
        where `id` = %s  
        and 
        `password` = %s
    """
result = sql_class.execute(sql,[_id,_pass])

corona_sql = """ 
            select * 
            from corona
         """

if result:
   ## 코로나 데이터 출력
   result2 = sql_class.excuteAll(corona_sql)
else:
    result2= print('로그인실패')

result2


,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,2022-06-08 09:09:05.982,24305,18188200,904,20220608,00:00,None,,
1,2022-06-07 09:09:00.897,24299,18174842,903,20220607,00:00,2022-06-08 09:10:36.846,,
2,2022-06-06 09:00:06.734,24279,18168670,902,20220606,00:00,2022-06-08 09:10:50.441,,
3,2022-06-05 08:53:19.426,24258,18163648,901,20220605,00:00,2022-06-08 09:11:04.758,,
4,2022-06-04 08:56:49.219,24238,18153814,900,20220604,00:00,2022-06-08 09:11:26.303,,
...,...,...,...,...,...,...,...,...,...
815,2020-03-14 00:00:00.000,72,8086,55,20200314,00:00,2021-10-07 10:30:51.51,261335,3.318000336
816,2020-03-13 00:00:00.000,67,7979,54,20200313,00:00,2021-10-07 10:30:51.51,248647,3.458499309
817,2020-03-12 00:00:00.000,66,7869,53,20200312,00:00,2021-10-07 10:30:51.51,234998,3.621744273
818,2020-03-11 00:00:00.000,60,7755,52,20200311,00:00,2021-10-07 10:30:51.51,222395,3.804174536


In [63]:
## 위의 코드를 함수 형태로 변환
#case1 
_id = input("id :")
_pass = input("password :")
sql = """ 
        select * 
        from user_list
        where
        `id` =%s
        and
        `password` = %s
        """

def login(input_sql,input_id,input_pass):
    result = sql_class.execute(input_sql,[input_id,input_pass])
    if result:
        corona_sql = """select * 
                        from corona
                        """
        return sql_class.excuteAll(corona_sql)
    else:
        return "로그인 실패"

login(sql,_id,_pass)

'로그인 실패'

In [66]:
# case2
def login2():
    _id = input("id:")
    _pass = input("password:")
    sql = """ 
            select *
            from user_list
            where `id` = %s and `password`=%s
            """
    result = sql_class.execute(sql,[_id , _pass])
    if result:
        corona_sql = """
                        select * 
                        from corona
                        """
        return sql_class.excuteAll(corona_sql)
    else:
        return "로그인 실패"

login2()

,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,2022-06-08 09:09:05.982,24305,18188200,904,20220608,00:00,None,,
1,2022-06-07 09:09:00.897,24299,18174842,903,20220607,00:00,2022-06-08 09:10:36.846,,
2,2022-06-06 09:00:06.734,24279,18168670,902,20220606,00:00,2022-06-08 09:10:50.441,,
3,2022-06-05 08:53:19.426,24258,18163648,901,20220605,00:00,2022-06-08 09:11:04.758,,
4,2022-06-04 08:56:49.219,24238,18153814,900,20220604,00:00,2022-06-08 09:11:26.303,,
...,...,...,...,...,...,...,...,...,...
815,2020-03-14 00:00:00.000,72,8086,55,20200314,00:00,2021-10-07 10:30:51.51,261335,3.318000336
816,2020-03-13 00:00:00.000,67,7979,54,20200313,00:00,2021-10-07 10:30:51.51,248647,3.458499309
817,2020-03-12 00:00:00.000,66,7869,53,20200312,00:00,2021-10-07 10:30:51.51,234998,3.621744273
818,2020-03-11 00:00:00.000,60,7755,52,20200311,00:00,2021-10-07 10:30:51.51,222395,3.804174536


### 연습문제
1. login() 함수를 변형
2. 매개변수가 3개에서 4개로 증가
3. sql, id, password와의 table매개변수 생성
4. table 매개변수 -> sql에 있는 table 이름
5. 예시 : table이 sales records이면  sales records데이터를 리턴
6. 대면 : 함수가 완성되면 sales records를 인자값에 넣어서 확인

In [13]:
_id = input("id :")
_pass = input("password :")
sql = """ 
        select * 
        from user_list
        where
        `id` =%s
        and
        `password` = %s
        """
_table = input("table : ")
def login(input_sql,input_id,input_pass,input_table):
    result = sql_class.execute(input_sql,[input_id,input_pass])
    if result:
        table_sql = f"""select * 
                        from `{input_table}`
                        """
        return sql_class.excuteAll(table_sql)
    else:
        return "로그인 실패"

login(sql,_id,_pass,_table)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Europe,Latvia,Beverages,Online,C,12/28/2015,361825549,1/23/2016,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Middle East and North Africa,Pakistan,Vegetables,Offline,C,1/13/2011,141515767,2/1/2011,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Sub-Saharan Africa,Democratic Republic of the Congo,Household,Online,C,9/11/2012,500364005,10/6/2012,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Europe,Czech Republic,Beverages,Online,C,10/27/2015,127481591,12/5/2015,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Asia,Laos,Beverages,Online,H,7/15/2014,199342048,7/31/2014,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Europe,Liechtenstein,Cosmetics,Online,C,10/27/2012,763044106,11/1/2012,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Sub-Saharan Africa,Democratic Republic of the Congo,Vegetables,Offline,M,2/14/2013,848579967,3/20/2013,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,Sub-Saharan Africa,South Africa,Meat,Online,L,2/19/2017,298185956,2/22/2017,8706,421.89,364.69,3672974.34,3174991.14,497983.20


### sales records 데이터를 전처리 과정
1. 총 수익, 총 비용, 총 이윤 파생변수 생성
2. 개수 * 가격 = 총 수익
3. 개수 * 원가 = 총 비용
4. 총 수익 - 총 비용 = 총 이윤


In [15]:
## 클래스 생성
sql_class = ms.Database("172.16.12.149",'ubion','1234')

In [18]:
sql = """
        select 
        `Country`as 국가,
        `Units Sold` as 판매개수,    
        `Unit Price` as 가격,
        `Unit Cost` as 원가,  
        (`Units Sold` * `Unit Price`) as 총수익,
        (`Units Sold` * `Unit Cost`) as 총비용,
        (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as 총이윤
        from 
        `sales records`
    """
sql_class.excuteAll(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


In [20]:
## 반올림
sql = """ 
    select 
    `Total Cost`,
    round(`Total Cost`,1) 
    from
    `sales records`
    """
sql_class.excuteAll(sql)

,Total Cost,"round(`Total Cost`,1)"
0,2353920.64,2353920.6
1,34174.25,34174.2
2,592408.95,592409.0
3,3861014.82,3861014.8
4,110978.89,110978.9
...,...,...
9995,273298.63,273298.6
9996,147991.46,147991.5
9997,229507.32,229507.3
9998,3174991.14,3174991.1


In [24]:
## 그룹화
## Country, Item Type 기준으로 그룹화
## Unit Sold 평균, 합계를 생성
## Country 오름차순정렬

sql = """ 
        select 
        `Country`,
        `Item Type`,
        AVG(`Units Sold`) as AVG,
        SUM(`Units Sold`) as SUM
        from `sales records`
        group by `Country`,`Item Type`
        order by `Country` ASC

    """
sql_class.excuteAll(sql)

,Country,Item Type,AVG,SUM
0,Afghanistan,Baby Food,1368.0000,1368
1,Afghanistan,Beverages,4793.3333,14380
2,Afghanistan,Cereal,6101.8889,54917
3,Afghanistan,Clothes,5977.4000,29887
4,Afghanistan,Cosmetics,3044.2500,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.0000,9640
2194,Zimbabwe,Office Supplies,4361.7500,17447
2195,Zimbabwe,Personal Care,9239.0000,18478
2196,Zimbabwe,Snacks,5438.0000,16314


### 부서이름이 `sales`인 소속 사람들의 정보를 출력
1. dept테이블에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번과정에서 나온 deptno를 기준으로 사원을 출력

In [35]:
sql = """ 
        select `DEPTNO`
        from dept
        where `DNAME`='SALES'
    """
sql_class.excuteAll(sql)

,DEPTNO
0,30.0


In [46]:
#case1

sql = """ 
        select * 
        from `emp`
        where `deptno` = 30
        """
sql_class.excuteAll(sql)

#case2
sql2 = """
        select * 
        from `emp`
        where `deptno` = (
            select `deptno`
            from dept
            where `dname` = 'sales'
        )
        """

sql_class.excuteAll(sql2)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
5,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0


In [79]:
### 부서 번호가 10 아니면 30인 경우의 사원의 정보를 출략  sales = 30  accounting =10
sql = """
        select * 
        from `emp`
        where `deptno` IN (
            select `deptno`
            from dept
            where `dname` in ('sales','accounting')
        )
        """
sql_class.excuteAll(sql)

sql2 = """ 
        select * 
        from `emp`
        where `deptno in (10,30)
    
        """
sql_class.excuteAll(sql2)


,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
1,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
2,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
3,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
4,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
5,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
6,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
7,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0
8,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10.0


In [83]:
sql = """select * from `emp`"""
df =sql_class.excuteAll(sql)
#case1
df[(df["DEPTNO"] ==30) | (df["DEPTNO"]==10)]
#case2
df[df["DEPTNO"].isin([10,30])]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
8,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0
11,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0
13,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10.0
